In [1]:
#pip install transformers
#pip install sentence_transformers

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, GPT2Model, CLIPTextModel, ElectraModel
import os
import tqdm

from sentence_transformers import SentenceTransformer, util

2023-09-29 12:16:58.713058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 12:16:59.543181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [4]:
def text_embeding(text, model, tokenizer):
    if model!= model_clip_ViT_B_32:
        inputs = tokenizer([text], return_tensors="pt")
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        embeddings = embeddings.tolist()[0]
    else:
        embeddings = model_clip_ViT_B_32.encode(text)
    return embeddings


In [5]:
'''distilbert = "distilbert-base-multilingual-cased"
tokenizer_distilbert = AutoTokenizer.from_pretrained(distilbert)
model_distilbert = AutoModel.from_pretrained(distilbert)

minilm = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer_minilm = AutoTokenizer.from_pretrained(minilm)
model_minilm = AutoModel.from_pretrained(minilm)

xtremedistil = "microsoft/xtremedistil-l6-h256-uncased"
tokenizer_xtremedistil = AutoTokenizer.from_pretrained(xtremedistil)
model_xtremedistil = AutoModel.from_pretrained(xtremedistil)

gpt2 = "gpt2"
tokenizer_gpt2 = AutoTokenizer.from_pretrained(gpt2)
model_gpt2 = GPT2Model.from_pretrained(gpt2)

electra = "google/electra-small-discriminator"
tokenizer_electra = AutoTokenizer.from_pretrained(electra)
model_electra = ElectraModel.from_pretrained(electra)

clip = "openai/clip-vit-base-patch32"
tokenizer_clip = AutoTokenizer.from_pretrained(clip)
model_clip = CLIPTextModel.from_pretrained(clip)'''

tokenizer_clip_ViT_B_32 = None
model_clip_ViT_B_32 = SentenceTransformer('clip-ViT-B-32')

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [6]:
'''
train = pd.read_csv('./coco/annotations/train_annotations.csv')
text = train['caption'].iloc[0]
text_embeding(text, model_clip, tokenizer_clip)
'''

"\ntrain = pd.read_csv('./coco/annotations/train_annotations.csv')\ntext = train['caption'].iloc[0]\ntext_embeding(text, model_clip, tokenizer_clip)\n"

In [7]:
train = pd.read_csv('./coco/annotations/train_annotations.csv')

In [8]:
# отбираем картинки, для которых были созданы эмбединги

img_encoding_path = './img_encodings/densenet'
emdeded_img = []

for f in files(path = img_encoding_path):
    path = img_encoding_path + "/" + f
    df = pd.read_parquet(path)
    imgs = list(df['img'])
    emdeded_img.append(imgs)


emdeded_img = sum(emdeded_img, [])

emdeded_img = list(set(emdeded_img))

train['embeded_img'] = train['image_id'].apply(lambda x: x in emdeded_img)

train = train[train.embeded_img == True]

In [9]:
def get_text_encoding_file(model,model_name,tokenizer,count_img):
    i = 0
    file_num = 0

    while i<=len(train):

        list_img = []
        list_ids = []
        list_encodings = []

        df = train.iloc[i:i+count_img]
        for j in tqdm.tqdm(range(len(df))):
            img = df['image_id'].iloc[j]
            id = df['id'].iloc[j]
            text = df['caption'].iloc[j]

            list_img.append(img)
            list_ids.append(id)
            list_encodings.append(text_embeding(text, model, tokenizer))

        df = pd.DataFrame(list_encodings)
        df['img'] = list_img
        df['id'] = list_ids
        df.columns = df.columns.map(str)

        file_name = f"./text_encodings/{model_name}/train_text_encodings_" + f"{model_name}" + "_" + f"{file_num}" + ".parquet"

        df.to_parquet(file_name)  

        file_num = file_num + 1
        i = i + count_img

In [10]:
count_img = 5003

In [11]:
model,model_name,tokenizer = model_clip_ViT_B_32, 'clip_ViT_B_32', tokenizer_clip_ViT_B_32
get_text_encoding_file(model,model_name,tokenizer,count_img)

100%|██████████| 5002/5002 [02:16<00:00, 36.70it/s]


In [12]:
'''model,model_name,tokenizer = model_distilbert, 'distilbert', tokenizer_distilbert
get_text_encoding_file(model,model_name,tokenizer,count_img)

model,model_name,tokenizer = model_minilm, 'minilm', tokenizer_minilm
get_text_encoding_file(model,model_name,tokenizer,count_img)

model,model_name,tokenizer = model_xtremedistil, 'xtremedistil', tokenizer_xtremedistil
get_text_encoding_file(model,model_name,tokenizer,count_img)

model,model_name,tokenizer = model_gpt2, 'gpt2', tokenizer_gpt2
get_text_encoding_file(model,model_name,tokenizer,count_img)

model,model_name,tokenizer = model_electra, 'electra', tokenizer_electra
get_text_encoding_file(model,model_name,tokenizer,count_img)

model,model_name,tokenizer = model_clip, 'clip', tokenizer_clip
get_text_encoding_file(model,model_name,tokenizer,count_img)'''

"model,model_name,tokenizer = model_distilbert, 'distilbert', tokenizer_distilbert\nget_text_encoding_file(model,model_name,tokenizer,count_img)\n\nmodel,model_name,tokenizer = model_minilm, 'minilm', tokenizer_minilm\nget_text_encoding_file(model,model_name,tokenizer,count_img)\n\nmodel,model_name,tokenizer = model_xtremedistil, 'xtremedistil', tokenizer_xtremedistil\nget_text_encoding_file(model,model_name,tokenizer,count_img)\n\nmodel,model_name,tokenizer = model_gpt2, 'gpt2', tokenizer_gpt2\nget_text_encoding_file(model,model_name,tokenizer,count_img)\n\nmodel,model_name,tokenizer = model_electra, 'electra', tokenizer_electra\nget_text_encoding_file(model,model_name,tokenizer,count_img)\n\nmodel,model_name,tokenizer = model_clip, 'clip', tokenizer_clip\nget_text_encoding_file(model,model_name,tokenizer,count_img)"

In [13]:
'''
restored_df = pd.read_parquet(path = './text_encodings/train_text_encodings_clip_28.parquet')
restored_df
'''

"\nrestored_df = pd.read_parquet(path = './text_encodings/train_text_encodings_clip_28.parquet')\nrestored_df\n"

In [17]:
restored_df = pd.read_parquet(path = './text_encodings/clip_ViT_B_32/train_text_encodings_clip_ViT_B_32_0.parquet')
restored_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,img,id
0,0.018809,0.146139,-0.115189,0.017918,0.189361,0.145464,-0.268781,-0.792693,-0.048998,-0.194030,...,0.534154,0.319716,-0.111667,-0.168765,-0.026466,0.057864,0.009861,-0.108892,6672,1132
1,-0.002951,0.110502,0.225368,0.109987,0.362135,0.076444,-0.089838,-0.762613,-0.237949,0.029581,...,0.599654,0.202179,-0.021066,-0.093879,-0.034572,0.085356,0.031636,-0.513478,6672,2155
2,0.071903,0.215447,-0.083460,-0.006173,-0.048616,-0.073526,-0.070855,-0.525942,-0.466571,-0.043207,...,0.502029,0.375081,0.042058,-0.342625,-0.321237,0.126255,0.160922,-0.444232,6672,3577
3,-0.168049,0.143850,0.153322,0.391258,-0.309038,0.149847,-0.473466,-0.251907,-0.380691,-0.230113,...,0.545995,0.348911,-0.318268,-0.173571,-0.046528,-0.106534,-0.059333,-0.365085,6672,5812
4,0.104517,-0.263003,0.053926,-0.268905,-0.365514,0.328480,-0.139194,-0.174823,-0.432008,-0.471242,...,0.107245,-0.277810,-0.502563,-0.259840,-0.320912,0.679533,0.332113,-0.245374,508605,10673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,0.256758,-0.083359,-0.275733,-0.237212,-0.285428,-0.173806,0.132147,-0.231798,-0.185969,0.383721,...,0.404989,-0.331715,0.097394,-0.361682,0.168551,0.131333,-0.117351,-0.574020,199554,795277
4998,0.038321,-0.162401,0.061246,-0.080975,0.161654,0.113905,-0.027144,-0.845765,-0.335149,0.066683,...,0.229448,-0.125402,-0.172273,-0.487198,-0.053185,-0.097777,-0.109612,-0.175229,199554,796174
4999,0.053171,-0.222858,0.178301,0.224710,0.339538,-0.238110,0.211407,-0.201101,-0.446005,-0.134897,...,1.286332,0.229796,0.159913,-0.069872,-0.175256,-0.048422,-0.193317,-0.185971,42751,818648
5000,0.149441,-0.214059,-0.683927,-0.018938,0.021045,0.340716,0.026361,-0.281545,-0.532534,0.051903,...,0.115421,0.128145,-0.060823,-0.035333,0.119270,0.240331,0.125112,0.093442,117774,825733


In [15]:
restored_df = pd.read_parquet(path = './text_encodings/electra/train_text_encodings_electra_0.parquet')
restored_df

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,img,id
0,0.043322,0.060295,-0.074070,0.019315,0.023831,-0.020827,0.059667,0.048850,0.067377,-0.034644,...,-0.007192,-0.054609,-0.062754,0.080716,-0.064543,0.052116,-0.054931,-0.008377,6672,1132
1,0.028191,0.027997,-0.028643,0.043141,0.026732,-0.031522,0.047419,0.079036,0.078998,-0.030068,...,0.020922,-0.045100,-0.041163,0.045660,-0.076239,0.060579,-0.060774,0.009789,6672,2155
2,0.005542,-0.008052,-0.087157,0.012577,-0.011111,-0.019439,0.118818,0.023921,0.066920,-0.010940,...,-0.030634,-0.057259,-0.092946,0.082850,-0.082470,0.092408,-0.087662,-0.046298,6672,3577
3,0.028694,0.041810,-0.055089,-0.042201,0.011262,-0.029541,0.085236,0.058430,0.030675,-0.015738,...,-0.033681,-0.041419,-0.008104,0.100445,-0.102710,0.093188,-0.094181,-0.000120,6672,5812
4,-0.012962,0.047918,-0.056447,-0.018373,-0.030927,-0.025824,0.129256,0.068601,0.065499,-0.036823,...,-0.044617,-0.016560,-0.056174,0.049475,-0.092387,0.130765,-0.019540,-0.071237,508605,10673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,0.024452,0.030419,-0.042686,-0.015397,0.027816,-0.026148,0.069040,0.051983,0.078553,-0.010842,...,0.002006,-0.051020,-0.007357,0.059570,-0.121622,0.071996,-0.077052,-0.007115,199554,795277
4998,0.012399,0.038470,-0.082469,-0.050223,0.000128,-0.019393,0.106340,0.028273,0.067827,-0.024491,...,-0.059383,-0.031438,-0.029288,0.094446,-0.084082,0.084933,-0.127071,-0.038647,199554,796174
4999,0.027242,0.038948,-0.040072,-0.005202,0.011536,0.006402,0.110860,0.081200,0.068212,-0.005035,...,-0.006187,-0.083230,-0.004159,0.095383,-0.140892,0.071558,-0.094642,-0.017444,42751,818648
5000,0.016898,0.048128,-0.027699,0.031465,0.043984,-0.004852,0.077365,0.077707,0.045920,-0.010357,...,0.021811,-0.047668,0.013100,0.054631,-0.086378,0.069427,-0.079738,-0.018114,117774,825733


In [16]:
restored_df = pd.read_parquet(path = './text_encodings/clip/train_text_encodings_clip_0.parquet')
restored_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,img,id
0,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,6672,1132
1,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,6672,2155
2,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,6672,3577
3,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,6672,5812
4,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,508605,10673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,199554,795277
4998,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,199554,796174
4999,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,42751,818648
5000,0.012348,0.004238,0.00371,0.001127,-0.960322,0.004808,0.004098,0.006641,0.009618,0.011742,...,0.000915,0.001635,0.003835,0.010812,0.011616,0.008981,0.021495,0.003687,117774,825733
